In [3]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('DS_Sample_1.csv')
df.shape

(1000, 18)

# 전처리

In [5]:
df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']].isna().sum()

CREDIT_LIMIT         0
MINIMUM_PAYMENTS    74
dtype: int64

In [8]:
df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']] =\
    df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']].apply(lambda x: x.fillna(value=x.mean()))

In [9]:
df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']].isna().sum()

CREDIT_LIMIT        0
MINIMUM_PAYMENTS    0
dtype: int64

In [10]:
card1 = df.copy()

# 1.

In [15]:
card1.groupby('TENURE')[['BALANCE', 'CREDIT_LIMIT']].corr()\
    .iloc[::2, -1].max().round(2)

0.95

# 2.

## 2-1.

In [17]:
df2 = card1.copy()

In [16]:
from sklearn.preprocessing import StandardScaler

In [19]:
col = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']
len(col)

17

In [21]:
scaler = StandardScaler().fit(df2[col])

In [24]:
df_scaled = pd.DataFrame(scaler.transform(df2[col]), columns=col)

In [26]:
df_scaled.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


## 2-2
# model / fit -> predict -> 평가

In [31]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [33]:
for n in [2, 3, 4, 5]:
    model = KMeans(n_clusters=n, random_state=1234).fit(df_scaled[col])
    pred = model.predict(df_scaled[col])
    print(n, silhouette_score(df_scaled[col], pred))

2 0.30752815304560793
3 0.19636128772937608
4 0.20715098494639902
5 0.19274056144483248


## 2-3.

In [34]:
model = KMeans(n_clusters=2, random_state=1234).fit(df_scaled[col])
pred = model.predict(df_scaled[col])

In [38]:
df2['cluster'] = pred

In [43]:
df2.groupby('cluster')['ONEOFF_PURCHASES'].mean().max().round(2)

3946.19

# 3.

## 3-1.

In [55]:
train = card1[card1['CUST_ID'] % 4 != 0]
test = card1[card1['CUST_ID'] % 4 == 0]

## 3-2.

In [56]:
card1.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

In [57]:
col = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']
len(col)

16

In [58]:
from sklearn.tree import DecisionTreeRegressor

In [59]:
model = DecisionTreeRegressor(random_state=1234)\
            .fit(X=train[col], y=train['ONEOFF_PURCHASES'])

## 3-3.

In [60]:
pred = model.predict(test[col])

In [61]:
from sklearn.metrics import mean_squared_error

In [63]:
round(mean_squared_error(y_true=test['ONEOFF_PURCHASES'], y_pred=pred) ** 0.5, 1)

2383.8